In [1]:
import arcpy
import os
import shutil
from arcpy.sa import Raster, SetNull, Int

# -----------------------------
# Setup environment
# -----------------------------
arcpy.env.workspace = r"C:\Users\ss2596\The University of Waikato\Lars Brabyn - Skye\00 Jake Overton\Njoko_Map\SENTINEL2 Mufunta\Mufunta_Sentinel2_aug_2025"
arcpy.env.overwriteOutput = True

# -----------------------------
# Sentinel-2 RGB bands
# -----------------------------
bands = {"B02": "Blue", "B03": "Green", "B04": "Red"}  # band codes
parent_folder = arcpy.env.workspace
temp_folder = os.path.join(parent_folder, "temp_tiffs")
os.makedirs(temp_folder, exist_ok=True)

mosaicked_band_files = []
threshold = 1000  # pixels below this value are considered invalid

# -----------------------------
# Collect SAFE folders alphabetically (priority)
# -----------------------------
safe_folders = sorted([f for f in os.listdir(parent_folder) if f.endswith(".SAFE")])
print(f"Found {len(safe_folders)} SAFE folders to process in priority order:")
for f in safe_folders:
    print("  ", f)

# -----------------------------
# Process each band
# -----------------------------
for band_code, band_name in bands.items():
    print(f"\nProcessing {band_name} ({band_code})...")

    tiff_list = []
    counter = 0

    for safe in safe_folders:
        granule_path = os.path.join(parent_folder, safe, "GRANULE")
        if not os.path.exists(granule_path):
            continue
        for granule_id in os.listdir(granule_path):
            img_data_path = os.path.join(granule_path, granule_id, "IMG_DATA", "R10m")
            if not os.path.exists(img_data_path):
                continue

            # find band JP2 (ignore mask JP2s)
            for f in os.listdir(img_data_path):
                if band_code in f and f.endswith(".jp2") and not f.startswith("MSK_"):
                    counter += 1
                    jp2_path = os.path.join(img_data_path, f)
                    tiff_path = os.path.join(temp_folder, os.path.basename(f).replace(".jp2", ".tif"))
                    arcpy.management.CopyRaster(jp2_path, tiff_path)

                    # -----------------------------
                    # Mask zeros and low-value pixels
                    # -----------------------------
                    band_r = Raster(tiff_path)
                    masked_r = SetNull((band_r < threshold) | (band_r == 0), band_r)

                    # Save final masked TIFF
                    masked_tiff = os.path.join(temp_folder, os.path.basename(f).replace(".jp2", "_masked.tif"))
                    masked_r.save(masked_tiff)
                    tiff_list.append(masked_tiff)

                    print(f"  [{counter}] Copied and masked: {f}")

    if not tiff_list:
        print(f"  ⚠️ No valid rasters found for {band_name}, skipping")
        continue

    # -----------------------------
    # Mosaic using BLEND (priority-aware)
    # -----------------------------
    out_raster = os.path.join(temp_folder, f"mosaic_{band_code}.tif")
    print(f"  Mosaicking {len(tiff_list)} rasters for {band_name} into {out_raster} with BLEND...")
    if len(tiff_list) > 1:
        arcpy.management.CopyRaster(tiff_list[0], out_raster)
        arcpy.management.Mosaic(
            inputs=tiff_list[1:],
            target=out_raster,
            mosaic_type="BLEND",
            nodata_value="0"
        )
    else:
        arcpy.management.CopyRaster(tiff_list[0], out_raster)

    mosaicked_band_files.append(out_raster)
    print(f"  ✔ {band_name} mosaic saved: {out_raster}")

# -----------------------------
# Convert to 8-bit and combine into final RGB raster
# -----------------------------
if mosaicked_band_files:
    arcpy.CheckOutExtension("Spatial")

    # Use parent folder name for final output
    folder_name = os.path.basename(parent_folder.rstrip("\\/"))
    final_rgb = os.path.join(parent_folder, f"{folder_name}_8bit.tif")

    print(f"\nScaling mosaicked bands to 8-bit and combining into final RGB raster: {final_rgb}...")

    scaled_bands = []
    for band_file in [os.path.join(temp_folder, f"mosaic_B04.tif"),  # Red
                      os.path.join(temp_folder, f"mosaic_B03.tif"),  # Green
                      os.path.join(temp_folder, f"mosaic_B02.tif")]: # Blue

        r = Raster(band_file)
        # Ignore 0/nodata pixels
        valid_r = SetNull(r == 0, r)
        min_val = float(arcpy.management.GetRasterProperties(valid_r, "MINIMUM").getOutput(0))
        max_val = float(arcpy.management.GetRasterProperties(valid_r, "MAXIMUM").getOutput(0))
        print(f"  Band {os.path.basename(band_file)}: min={min_val}, max={max_val}")

        # Scale to 0-255 and convert to integer
        scaled = Int(((r - min_val) / (max_val - min_val)) * 255)
        scaled_bands.append(scaled)

    # Combine scaled bands into 8-bit RGB
    arcpy.management.CompositeBands(scaled_bands, final_rgb)

    # Calculate statistics
    arcpy.management.CalculateStatistics(final_rgb)
    print(f"\n✅ Final 8-bit RGB mosaic saved at: {final_rgb}")

# -----------------------------
# Delete Temporary Files
# -----------------------------
print(f"\nCleaning up temporary files in: {temp_folder} ...")
del band_r, masked_r
arcpy.ClearEnvironment("workspace")

import gc, time
gc.collect()
time.sleep(2)  # ensure file handles are released

try:
    shutil.rmtree(temp_folder)
    print("✅ Temporary folder removed successfully.")
except Exception as e:
    print(f"⚠️ Could not remove temp folder: {e}")


Found 6 SAFE folders to process in priority order:
   S2B_MSIL2A_20250822T081609_N0511_R121_T35LKC_20250822T105706.SAFE
   S2B_MSIL2A_20250822T081609_N0511_R121_T35LKD_20250822T105706.SAFE
   S2B_MSIL2A_20250822T081609_N0511_R121_T35LLC_20250822T105706.SAFE
   S2B_MSIL2A_20250822T081609_N0511_R121_T35LLD_20250822T105706.SAFE
   S2C_MSIL2A_20250824T080631_N0511_R078_T35LLC_20250824T133818.SAFE
   S2C_MSIL2A_20250824T080631_N0511_R078_T35LLD_20250824T133818.SAFE

Processing Blue (B02)...
  [1] Copied and masked: T35LKC_20250822T081609_B02_10m.jp2
  [2] Copied and masked: T35LKD_20250822T081609_B02_10m.jp2
  [3] Copied and masked: T35LLC_20250822T081609_B02_10m.jp2
  [4] Copied and masked: T35LLD_20250822T081609_B02_10m.jp2
  [5] Copied and masked: T35LLC_20250824T080631_B02_10m.jp2
  [6] Copied and masked: T35LLD_20250824T080631_B02_10m.jp2
  Mosaicking 6 rasters for Blue into C:\Users\ss2596\The University of Waikato\Lars Brabyn - Skye\00 Jake Overton\Njoko_Map\SENTINEL2 Mufunta\Mufunta